In [30]:
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt

In [33]:
# Ścieżka do pliku CSV
file_path = "./mushroom-classification/mushrooms.csv"

# Wczytaj dane
X = pd.read_csv(file_path)
# Wyświetl pierwsze kilka wierszy
y = data["class"]
#print(y.unique())
#for feature in X.columns:
#print(y)
#print(X['bruises'].unique())
#X_encoded = X.apply(lambda col: col.astype('category').cat.codes)
#y_encoded = y.astype('category').cat.codes
for col, value in X.iloc[1].items():
    print(f"Kolumna: {col}, Wartość: {value}")

Kolumna: class, Wartość: e
Kolumna: cap-shape, Wartość: x
Kolumna: cap-surface, Wartość: s
Kolumna: cap-color, Wartość: y
Kolumna: bruises, Wartość: t
Kolumna: odor, Wartość: a
Kolumna: gill-attachment, Wartość: f
Kolumna: gill-spacing, Wartość: c
Kolumna: gill-size, Wartość: b
Kolumna: gill-color, Wartość: k
Kolumna: stalk-shape, Wartość: e
Kolumna: stalk-root, Wartość: c
Kolumna: stalk-surface-above-ring, Wartość: s
Kolumna: stalk-surface-below-ring, Wartość: s
Kolumna: stalk-color-above-ring, Wartość: w
Kolumna: stalk-color-below-ring, Wartość: w
Kolumna: veil-type, Wartość: p
Kolumna: veil-color, Wartość: w
Kolumna: ring-number, Wartość: o
Kolumna: ring-type, Wartość: p
Kolumna: spore-print-color, Wartość: n
Kolumna: population, Wartość: n
Kolumna: habitat, Wartość: g


In [64]:
class MulitnomialNaiveBayesClassifie:
    def __init__(self):
        self.feature_probability={}
        self.conditional_probability={} #domyslnie kazde jest warunkowe wzgldem wartosci z feature_probability.

    def fit(self, X_train, column_name):
        """
        X_train jest tablica wartosci w naszym zbiorze, zapisany z uzyciem DataFrame'u, bibliotek pandas.
        column_name, to nazwa kolumn z X_train wzgledem, ktorej dokuonujemy estymacji.
        Nie zwraca nic.
        """
        y=X_train[column_name]
        for feature in y.unique():
            self.feature_probability[feature]=len(y[y==feature])/len(y) #mozna poprawc, by przechowywac liczby calkowito liczbowe

        for column in X.columns:
            if column == column_name:
                continue
            self.conditional_probability[column]={}
            for conditional_feature in X[column].unique():
                self.conditional_probability[column][conditional_feature]={}
                for feature in y.unique():
                    feature_count=len(y[y==feature])
                    conditional_feature_count=len(X_train[(y==feature) & (X_train[column]==conditional_feature)])
                    self.conditional_probability[column][conditional_feature][feature]=conditional_feature_count/feature_count
                    #dodac sytuacje do braku wartosci: "?"
    
    def predict_option(self, row_to_predict, column_name, b):    
        probability_per_feature={}
        # Obliczanie prawdopodobieństwa dla każdej klasy
        for feature in self.feature_probability:
            probability = self.feature_probability[feature]

            for column, value in row_to_predict.items():
                
                if column == column_name:
                    continue  
                elif value in self.conditional_probability[column]:
                    conditional_probability_now=self.conditional_probability[column][value].get(feature, 0)
                    
                    if not conditional_probability_now:
                        conditional_probability_now = 1e-16
                        
                    probability *= conditional_probability_now
                else:
                    print("zero")
                    probability *= 1e-16  # Brak warunku oznacza male prawdopodobienstwo, nie moze byc zero, ale jest bliskie zeru.  
            probability_per_feature[feature] = probability

        # Wybieranie klasy z najwyższym prawdopodobieństwem
        if b:
            predicted_feature = max(probability_per_feature, key=probability_per_feature.get)
            return predicted_feature
        else:
            return list(probability_per_feature.items())
        
    def predict(self, row_to_predict, column_name):
        """
        row_to_predict: struktura Seris zawierający dane, dla których chcemy wykonać predykcję
        """
        return self.predict_option(row_to_predict, column_name, True)
        
    def predict_proba(self, row_to_predict, column_name):
        return self.predict_option(row_to_predict, column_name, False)

In [65]:
"""
tworzenie klasyfikatora: NBC=MulitnomialNaiveBayesClassifie()
przygotowywanie zbioru testowego i treningowego: X_train, X_test = train_test_split(X, test_size=0.3, random_state=42), test_size oznacza rozmiar probki, aktualnie 70% jest do trenowania systemu, a pozostale do testow
trenowanie systemu: NBC.fit(X_train, "nazwa_kolumny_do_nauki"), dla nas nazwa... powinna być "class", wszystkie nazwy kolumn: X.columns
estymacja dla p o j e d y n c z e g o wiersza NBC.predict(X_test.iloc[ind]), gdzie ind to numer wiersza, liczac od gory, zwraca najbardziej prawdopodobna wartosc zmiennej szukanej.
tablica prawdopodobienstw, mozliwych wynikow, dla p o j e d y n c z e g o wiersza: NBC.predict_proba(X_test.iloc[ind]) 
"""
NBC=MulitnomialNaiveBayesClassifie()
NBC.fit(X, "class")
NBC.predict_proba(X.iloc[1], "class") 

[('p', 4.866581859441924e-44), ('e', 2.2635937728415586e-09)]

In [ ]:
for column in data.columns:
    if column != "class": # Ominięcie kolumny z klasą
        print(f"Cecha: {column}")
        print(data.groupby("class")[column].value_counts(), "\n")

In [4]:

# Wykres punktowy zapachu (odor) i rozmiaru blaszek (gill-size)
sns.scatterplot(data=data_encoded, x='odor', y='gill-size', hue=data['class'], palette={'e': 'green', 'p': 'red'})
plt.title("Zależność między 'odor' a 'gill-size'")
plt.show()

NameError: name 'sns' is not defined